In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import drive
from google.colab import files

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Datathon/df')
df.head(10)

id       fecha comercio   monto  \
0  91477f382c3cf63ab5cd9263b502109243741158  2022-01-02   AMAZON    5.99   
1  91477f382c3cf63ab5cd9263b502109243741158  2022-01-05    RAPPI   13.01   
2  91477f382c3cf63ab5cd9263b502109243741158  2022-01-05    RAPPI   15.84   
3  91477f382c3cf63ab5cd9263b502109243741158  2022-01-05   AMAZON    8.17   
4  91477f382c3cf63ab5cd9263b502109243741158  2022-01-05   AMAZON    2.54   
5  91477f382c3cf63ab5cd9263b502109243741158  2022-01-06   AMAZON    5.20   
6  91477f382c3cf63ab5cd9263b502109243741158  2022-01-07   AMAZON   16.33   
7  91477f382c3cf63ab5cd9263b502109243741158  2022-01-11      CFE  185.78   
8  91477f382c3cf63ab5cd9263b502109243741158  2022-01-13   AMAZON   13.91   
9  91477f382c3cf63ab5cd9263b502109243741158  2022-01-14   AMAZON   19.66   

   tipo_Venta_binaria  id_municipio  
0                   1     6519019.0  
1                   1     6519019.0  
2                   1     6519019.0  
3                   1     6519019.0  
4                   1     6519019.0  
5                   1     6519019.0  
6                   1     6519019.0  
7                   1     6519019.0  
8                   1     6519019.0  
9                   1     6519019.0

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343979 entries, 0 to 343978
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  343979 non-null  object 
 1   fecha               343979 non-null  object 
 2   comercio            343979 non-null  object 
 3   monto               343979 non-null  float64
 4   tipo_Venta_binaria  343979 non-null  int64  
 5   id_municipio        343979 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 15.7+ MB


In [5]:
df.describe()

monto  tipo_Venta_binaria  id_municipio
count  343979.000000       343979.000000  3.439790e+05
mean       42.288464            0.681731  6.220330e+06
std        94.614261            0.465805  6.918740e+05
min         1.390000            0.000000  4.701001e+06
25%         9.320000            0.000000  5.509016e+06
50%        17.930000            1.000000  6.519006e+06
75%        39.460000            1.000000  6.519046e+06
max      2297.880000            1.000000  7.832056e+06

In [6]:
df.describe(include='object')

id       fecha comercio
count                                     343979      343979   343979
unique                                       995         395       97
top     155a6724b044c00b762c334737124fe4602cf0eb  2022-02-28     UBER
freq                                        2462        1614    40174

In [9]:
df.shape

(343979, 6)

In [10]:
df['comercio'].unique()

array(['AMAZON', 'RAPPI', 'CFE', 'OXXO', 'FARMACIAS DEL AHORRO',
       '7 ELEVEN', 'SORIANA', 'HEB', 'LIVERPOOL', 'SPOTIFY',
       'SUPERCENTER', 'COSTCO', 'OXXO GAS', 'UBER', 'STARBUCKS',
       'UBER EATS', 'SERV AGUA DREN', 'AMAZON PRIME', 'UNDOSTRES',
       'MERCADO PAGO', 'RAPPIPRO', 'WALMART', 'TELCEL', 'TELMEX',
       'CRUNCHYROLL', 'FARMACIAS GUADALAJARA', 'IZZI', 'CINEPOLIS',
       'DISNEY PLUS', 'BAIT', 'NAYAX', 'CARLS JR', 'APPLE', 'TOTALPLAY',
       'SAMS CLUB', 'NETFLIX', 'MELIMAS', 'MICROSOFT', 'MAX', 'TULOTERO',
       'BAE', 'DIDI RIDES', 'GOOGLE YOUTUBEPREMIUM', 'ALLIANZ MEXICO',
       'TOTAL PLAY', 'ITUNES', 'SMART', 'ALSUPER', 'UBRPAGOSMEX', 'PARCO',
       'FACEBOOK', '7ELEVEN', 'TEMU', 'CHEDRAUI', 'COPPEL',
       'FARMACIAS SIMILARES', 'GOOGLE', 'MI ATT', 'SUPERAMA', 'ROTOPLAS',
       'WAL-MART', 'MERCADOPAGO', 'TOTAL PASS', 'ALIEXPRESS', 'CALIENTE',
       'DIDI', 'DIDI FOOD', 'CASHI ECOMMERCE', 'DIDIFOOD', 'GOOGLE ONE',
       'APLAZO', 'APLAZ', 'SOFT RA

In [11]:
df['tipo_Venta_binaria'].unique()

array([1, 0])

In [20]:
# Series requeridas por cliente
por_cliente = df.groupby('id')

# 1. Series de fechas de compra ordenadas cronológicamente
fechas_cliente = por_cliente['fecha'].apply(list).apply(sorted)

# 2. Series de montos correspondientes a cada fecha
montos_cliente = por_cliente.apply(
    lambda x: x.sort_values('fecha')['monto'].values,
    include_groups=False
)

# 3. Diferencias temporales entre compras consecutivas (∆t)
delta_t_cliente = por_cliente.apply(
    lambda x: pd.to_datetime(x.sort_values('fecha')['fecha']).diff().dt.days.fillna(0).values[1:],
    include_groups=False
)

# 4. Tipo de venta (binario) por fecha
tipo_venta_cliente = por_cliente.apply(
    lambda x: x.sort_values('fecha')['tipo_Venta_binaria'].values,
    include_groups=False
)

# 5. Comercios por fecha
comercio_cliente = por_cliente.apply(
    lambda x: x.sort_values('fecha')['comercio'].values,
    include_groups=False
)

# Para análisis temporal (ventana móvil)
tau_meses = 90  # días
delta_t_ventana = por_cliente.apply(
    lambda x: pd.to_datetime(x.sort_values('fecha')['fecha']).diff().dt.days.values[-tau_meses:]
    if len(x) > 1 else [],
    include_groups=False
)

# Variables para modelado multivariado
variables_multivariadas = por_cliente.apply(
    lambda x: pd.DataFrame({
        'delta_t': pd.to_datetime(x.sort_values('fecha')['fecha']).diff().dt.days.fillna(0).values[1:],
        'monto': x.sort_values('fecha')['monto'].values[1:],
        'tipo_venta': x.sort_values('fecha')['tipo_Venta_binaria'].values[1:],
        'comercio': x.sort_values('fecha')['comercio'].values[1:]
    }) if len(x) > 1 else pd.DataFrame(),
    include_groups=False
)

# Series para predicción probabilística
intensidad_compra = por_cliente.apply(
    lambda x: pd.to_datetime(x.sort_values('fecha')['fecha']).values,
    include_groups=False
)

montos_distribucion = por_cliente['monto'].apply(list)

# Series para actualización online
series_historicas = por_cliente.apply(
    lambda x: {
        'fechas': pd.to_datetime(x.sort_values('fecha')['fecha']).values,
        'montos': x.sort_values('fecha')['monto'].values,
        'tipo_venta': x.sort_values('fecha')['tipo_Venta_binaria'].values,
        'comercio': x.sort_values('fecha')['comercio'].values
    },
    include_groups=False
)

# Series para tendencias temporales (filtro Kalman)
series_tendencias = por_cliente.apply(
    lambda x: pd.DataFrame({
        'fecha': pd.to_datetime(x.sort_values('fecha')['fecha']),
        'delta_t': pd.to_datetime(x.sort_values('fecha')['fecha']).diff().dt.days,
        'monto': x.sort_values('fecha')['monto']
    }).set_index('fecha').resample('W').mean().ffill(),
    include_groups=False
)

# Series para análisis de tendencia comercial - CORREGIDO
tendencia_comercio = por_cliente.apply(
    lambda x: pd.DataFrame({
        'fecha': pd.to_datetime(x.sort_values('fecha')['fecha']),
        'comercio': x.sort_values('fecha')['comercio']
    }).set_index('fecha').groupby('comercio').resample('ME').count(),  # 'ME' en lugar de 'M'
    include_groups=False
)

# Series para análisis de estacionalidad
estacionalidad = por_cliente.apply(
    lambda x: pd.DataFrame({
        'fecha': pd.to_datetime(x.sort_values('fecha')['fecha']),
        'monto': x.sort_values('fecha')['monto']
    }).set_index('fecha').assign(
        dia_semana=lambda df: df.index.dayofweek,
        mes=lambda df: df.index.month
    ),
    include_groups=False
)

# Series para análisis de valores atípicos (outliers)
outliers_monto = por_cliente['monto'].apply(
    lambda x: x[abs(x - x.mean()) > 2*x.std()] if len(x) > 1 else pd.Series()
)

# Series para análisis de velocidad de gasto
velocidad_gasto = por_cliente.apply(
    lambda x: pd.DataFrame({
        'fecha': pd.to_datetime(x.sort_values('fecha')['fecha']),
        'monto_acumulado': x.sort_values('fecha')['monto'].cumsum(),
        'dias_acumulados': range(len(x))
    }) if len(x) > 0 else pd.DataFrame(),
    include_groups=False
)

In [22]:
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=c5b88f7c0167a26971d0713bd10d536159e0272d7ea3ee9e35657fa08062ea14
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy


In [24]:
!pip install copulas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.0 MB/s eta 0:00:00


In [26]:
!pip install tslearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 6.1 MB/s eta 0:00:00


In [27]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.mixture import GaussianMixture
from hdbscan import HDBSCAN
from scipy.stats import gamma
import statsmodels.api as sm
from filterpy.kalman import KalmanFilter
from copulas.multivariate import GaussianMultivariate
from tslearn.metrics import dtw

class AlgoritmoPrediccionCompras:
    def __init__(self, df):
        """
        Inicializar el algoritmo con el DataFrame de transacciones

        Args:
            df: DataFrame con columnas id, fecha, comercio, monto, tipo_Venta_binaria, id_municipio
        """
        self.df = df
        self.clientes = df['id'].unique()
        self.por_cliente = df.groupby('id')
        self.modelos_cliente = {}

    def ejecutar_para_todos(self):
        """Ejecutar algoritmo completo para todos los clientes"""
        for cliente in self.clientes:
            self.ejecutar_para_cliente(cliente)

    def ejecutar_para_cliente(self, cliente_id):
        """
        Ejecutar algoritmo completo para un cliente específico

        Args:
            cliente_id: Identificador único del cliente
        """
        # Obtener datos del cliente
        datos_cliente = self.df[self.df['id'] == cliente_id].sort_values('fecha')

        if len(datos_cliente) <= 1:
            print(f"Cliente {cliente_id} tiene muy pocas transacciones para análisis")
            return None

        # 1. Preprocesamiento
        datos_procesados = self.preprocesamiento(datos_cliente)

        # 2. Análisis temporal
        clusters = self.analisis_temporal(datos_procesados)

        # 3. Modelado multivariado
        gmm, subpatrones = self.modelado_multivariado(datos_procesados)

        # 4. Predicción probabilística
        prediccion = self.prediccion_probabilistica(datos_procesados, gmm, subpatrones)

        # 5. Guardar modelo para actualizaciones futuras
        self.modelos_cliente[cliente_id] = {
            'gmm': gmm,
            'subpatrones': subpatrones,
            'datos_procesados': datos_procesados,
            'prediccion': prediccion
        }

        return prediccion

    def preprocesamiento(self, datos_cliente):
        """
        Paso 1: Preprocesamiento de los datos

        Args:
            datos_cliente: DataFrame con datos de un solo cliente
        """
        # Convertir fechas a datetime y ordenar
        datos_cliente = datos_cliente.copy()
        datos_cliente['fecha'] = pd.to_datetime(datos_cliente['fecha'])
        datos_cliente = datos_cliente.sort_values('fecha')

        # Suavizado temporal LOWESS para fechas de compra
        fechas_num = (datos_cliente['fecha'] - datos_cliente['fecha'].min()).dt.days.values
        fechas_suavizadas = sm.nonparametric.lowess(
            fechas_num,
            np.arange(len(fechas_num)),
            frac=0.3
        )[:, 1]

        # Convertir fechas suavizadas de vuelta a datetime
        datos_cliente['fecha_suavizada'] = datos_cliente['fecha'].min() + pd.to_timedelta(fechas_suavizadas, unit='D')

        # Calcular diferencias temporales entre compras consecutivas
        datos_cliente['delta_t'] = datos_cliente['fecha'].diff().dt.days.fillna(0)

        # Detección de outliers con Isolation Forest
        if len(datos_cliente) > 10:  # Necesitamos suficientes datos para el modelo
            clf = IsolationForest(contamination=0.1, random_state=42)
            datos_outlier = np.column_stack([
                datos_cliente['delta_t'].values[1:],
                datos_cliente['monto'].values[1:]
            ])
            if len(datos_outlier) > 1:
                datos_cliente['outlier_score'] = [0] + list(clf.fit_predict(datos_outlier))
                # Marcar outliers (valores negativos son outliers)
                datos_cliente['es_outlier'] = datos_cliente['outlier_score'] < 0
            else:
                datos_cliente['es_outlier'] = False
        else:
            datos_cliente['es_outlier'] = False

        return datos_cliente

    def analisis_temporal(self, datos_procesados, tau_dias=90):
        """
        Paso 2: Análisis temporal con clustering

        Args:
            datos_procesados: DataFrame con datos preprocesados
            tau_dias: Ventana móvil de días para análisis
        """
        # Extraer datos temporales
        fechas = datos_procesados['fecha'].values
        montos = datos_procesados['monto'].values
        delta_t = datos_procesados['delta_t'].values

        # Si hay muy pocos datos, no hacer clustering
        if len(fechas) < 5:
            return {'clusters': np.zeros(len(fechas))}

        # Preparar características para clustering
        # Usar DTW (Dynamic Time Warping) para medir distancias
        # Aquí simplificamos usando las características directamente
        features = np.column_stack([delta_t[1:], montos[1:]])

        # Clusterizar con HDBSCAN si hay suficientes datos
        if len(features) >= 5:
            try:
                clusterer = HDBSCAN(min_cluster_size=2, min_samples=1, metric='euclidean')
                clusters = clusterer.fit_predict(features)
                # Añadir un 0 al inicio para mantener la misma longitud
                clusters_full = np.append([0], clusters)
            except Exception as e:
                print(f"Error en clustering HDBSCAN: {e}")
                clusters_full = np.zeros(len(fechas))
        else:
            clusters_full = np.zeros(len(fechas))

        return {'clusters': clusters_full}

    def modelado_multivariado(self, datos_procesados, n_components=2):
        """
        Paso 3: Modelado multivariado con GMM y Cópulas

        Args:
            datos_procesados: DataFrame con datos preprocesados
            n_components: Número de componentes para GMM
        """
        # Extraer características para el modelo
        if len(datos_procesados) <= 1:
            return None, None

        # Preparar datos para GMM
        features = np.column_stack([
            datos_procesados['delta_t'].values[1:],
            datos_procesados['monto'].values[1:],
            datos_procesados['tipo_Venta_binaria'].values[1:]
        ])

        # Entrenar GMM si hay suficientes datos
        if len(features) >= n_components + 1:
            gmm = GaussianMixture(
                n_components=min(n_components, len(features) - 1),
                covariance_type='full',
                random_state=42
            )
            gmm.fit(features)

            # Extraer subpatrones de compra
            subpatrones = []
            for i in range(gmm.n_components):
                # Medias para delta_t, monto
                mu_delta_t = gmm.means_[i, 0]
                mu_monto = gmm.means_[i, 1]

                # Moda para tipo_venta (simplificación, usar la media como aproximación)
                tau_mode = round(gmm.means_[i, 2])

                subpatrones.append({
                    'mu_delta_t': mu_delta_t,
                    'mu_monto': mu_monto,
                    'tau_mode': tau_mode,
                    'peso': gmm.weights_[i]
                })

            # Para modelar dependencias no lineales con cópulas
            # (simplificado para este ejemplo)
            try:
                copula = GaussianMultivariate()
                copula.fit(pd.DataFrame({
                    'delta_t': datos_procesados['delta_t'].values[1:],
                    'monto': datos_procesados['monto'].values[1:]
                }))
            except Exception as e:
                print(f"Error en ajuste de cópula: {e}")
                copula = None

            return gmm, {'subpatrones': subpatrones, 'copula': copula}
        else:
            return None, None

    def prediccion_probabilistica(self, datos_procesados, gmm, subpatrones):
        """
        Paso 4: Predicción probabilística con proceso Hawkes y Gamma

        Args:
            datos_procesados: DataFrame con datos preprocesados
            gmm: Modelo GMM entrenado
            subpatrones: Diccionario con subpatrones extraídos del GMM
        """
        if gmm is None or len(datos_procesados) < 3:
            # No podemos hacer predicciones con datos insuficientes
            return {
                'siguiente_fecha': None,
                'siguiente_monto': None,
                'siguiente_tipo': None
            }

        # 1. Predicción de tiempo con proceso Hawkes
        fechas = pd.to_datetime(datos_procesados['fecha']).values
        ultima_fecha = fechas[-1]

        # Parámetros simplificados del proceso Hawkes
        mu = 1/datos_procesados['delta_t'].mean() if datos_procesados['delta_t'].mean() > 0 else 0.1
        alpha = 0.8  # Influencia de eventos pasados
        beta = 0.2   # Tasa de decaimiento

        # Simulación para estimar la próxima fecha (enfoque simplificado)
        dias_predichos = []
        for _ in range(100):  # Simular 100 veces
            # Tiempo hasta el próximo evento en días
            tiempo_siguiente = np.random.exponential(1/mu)
            dias_predichos.append(tiempo_siguiente)

        dias_hasta_siguiente = np.median(dias_predichos)  # Usar la mediana como estimador robusto
        siguiente_fecha = ultima_fecha + pd.Timedelta(days=dias_hasta_siguiente)

        # 2. Predicción de monto con distribución Gamma
        montos = datos_procesados['monto'].values
        mu_a = montos.mean()
        sigma_a = montos.std()

        # Parámetros de la distribución Gamma
        k = (mu_a**2) / (sigma_a**2) if sigma_a > 0 else 1
        theta = (sigma_a**2) / mu_a if mu_a > 0 else 1

        # Generar predicción de monto
        siguiente_monto = gamma.rvs(k, scale=theta)

        # 3. Predicción de tipo de venta
        # Usar el subpatrón con mayor probabilidad
        if subpatrones is not None and 'subpatrones' in subpatrones:
            mejor_subpatron = max(subpatrones['subpatrones'], key=lambda x: x['peso'])
            siguiente_tipo = int(mejor_subpatron['tau_mode'])
        else:
            # Si no hay subpatrones, usar el tipo más frecuente
            siguiente_tipo = int(datos_procesados['tipo_Venta_binaria'].mode()[0])

        return {
            'siguiente_fecha': siguiente_fecha,
            'siguiente_monto': siguiente_monto,
            'siguiente_tipo': siguiente_tipo
        }

    def actualizar_online(self, cliente_id, nueva_compra):
        """
        Paso 5: Actualización online bayesiana

        Args:
            cliente_id: ID del cliente para actualizar su modelo
            nueva_compra: Dict con fecha, monto y tipo_venta de la nueva compra
        """
        if cliente_id not in self.modelos_cliente:
            print(f"No existe modelo para el cliente {cliente_id}")
            return

        modelo = self.modelos_cliente[cliente_id]
        datos_procesados = modelo['datos_procesados']

        # Añadir la nueva compra a los datos procesados
        nueva_fila = pd.DataFrame({
            'id': [cliente_id],
            'fecha': [pd.to_datetime(nueva_compra['fecha'])],
            'monto': [nueva_compra['monto']],
            'tipo_Venta_binaria': [nueva_compra['tipo_venta']],
            'comercio': [nueva_compra.get('comercio', 'DESCONOCIDO')],
            'id_municipio': [datos_procesados['id_municipio'].iloc[0]]  # Usar el mismo municipio
        })

        # Actualizar el dataframe con la nueva compra
        datos_actualizados = pd.concat([datos_procesados, nueva_fila], ignore_index=True)
        datos_actualizados = datos_actualizados.sort_values('fecha')

        # Reprocesar con los nuevos datos
        datos_procesados_nuevos = self.preprocesamiento(datos_actualizados)

        # Actualizar modelo GMM (enfoque simplificado)
        # En la práctica se usaría un enfoque bayesiano más complejo
        gmm_nuevo, subpatrones_nuevos = self.modelado_multivariado(datos_procesados_nuevos)

        # Implementar filtro de Kalman para seguimiento de tendencias
        if 'kf' not in modelo:
            # Inicializar filtro de Kalman si no existe
            kf = KalmanFilter(dim_x=2, dim_z=1)  # Estados: [media_delta_t, std_delta_t], Obs: delta_t
            kf.x = np.array([[datos_procesados['delta_t'].mean()],
                             [datos_procesados['delta_t'].std()]])  # Estado inicial
            kf.F = np.array([[1., 0.],
                            [0., 1.]])  # Matriz de transición
            kf.H = np.array([[1., 0.]])  # Matriz de observación
            kf.P *= 1000.  # Covarianza inicial
            kf.R = np.array([[5.]])  # Covarianza de medición
            kf.Q = np.array([[0.1, 0.],
                             [0., 0.1]])  # Covarianza de proceso
            modelo['kf'] = kf
        else:
            kf = modelo['kf']

        # Calcular delta_t para la nueva compra
        nueva_delta_t = (pd.to_datetime(nueva_compra['fecha']) -
                          datos_procesados['fecha'].iloc[-1]).days

        # Actualizar filtro de Kalman con la nueva observación
        kf.predict()
        kf.update(np.array([nueva_delta_t]))

        # Actualizar el modelo del cliente
        self.modelos_cliente[cliente_id] = {
            'gmm': gmm_nuevo if gmm_nuevo is not None else modelo['gmm'],
            'subpatrones': subpatrones_nuevos if subpatrones_nuevos is not None else modelo['subpatrones'],
            'datos_procesados': datos_procesados_nuevos,
            'kf': kf,
            'prediccion': self.prediccion_probabilistica(
                datos_procesados_nuevos,
                gmm_nuevo if gmm_nuevo is not None else modelo['gmm'],
                subpatrones_nuevos if subpatrones_nuevos is not None else modelo['subpatrones']
            )
        }

        return self.modelos_cliente[cliente_id]['prediccion']

In [28]:
# Instanciar y ejecutar el algoritmo
algoritmo = AlgoritmoPrediccionCompras(df)

# Ejecutar para todos los clientes
algoritmo.ejecutar_para_todos()

# Alternativamente, ejecutar para un cliente específico
cliente_id = df['id'].iloc[0]
prediccion = algoritmo.ejecutar_para_cliente(cliente_id)
print(f"Predicción para cliente {cliente_id}:")
print(f"Siguiente compra: {prediccion['siguiente_fecha']}")
print(f"Monto estimado: ${prediccion['siguiente_monto']:.2f}")
print(f"Tipo de venta: {'Online' if prediccion['siguiente_tipo'] == 1 else 'Física'}")

# Simular una nueva compra y actualizar modelo
nueva_compra = {
    'fecha': '2022-03-15',
    'monto': 25.50,
    'tipo_venta': 1,  # Online
    'comercio': 'AMAZON'
}
nueva_prediccion = algoritmo.actualizar_online(cliente_id, nueva_compra)
print(f"\nPredicción actualizada tras nueva compra:")
print(f"Siguiente compra: {nueva_prediccion['siguiente_fecha']}")
print(f"Monto estimado: ${nueva_prediccion['siguiente_monto']:.2f}")
print(f"Tipo de venta: {'Online' if nueva_prediccion['siguiente_tipo'] == 1 else 'Física'}")

Se han truncado las últimas 5000 líneas del flujo de salida.
/usr/local/lib/python3.11/dist-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.11/dist-packages/scipy/stats/_continuous_distns.py:800: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  a, b = optimize.fsolve(func, (1.0, 1.0))
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning: invalid value enco

Predicción para cliente 91477f382c3cf63ab5cd9263b502109243741158:
Siguiente compra: 2023-01-28 18:03:36.610633692
Monto estimado: $2.20
Tipo de venta: Online


/usr/local/lib/python3.11/dist-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.11/dist-packages/scipy/stats/_continuous_distns.py:800: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  a, b = optimize.fsolve(func, (1.0, 1.0))
/usr/local/lib/python3.11/dist-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.11/dist-packages/scipy/stats/_continuous_distns.py:800: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  a, b = optimize.fsolve(func, (1.0, 1.0))



Predicción actualizada tras nueva compra:
Siguiente compra: 2023-01-28 23:59:16.118920920
Monto estimado: $0.00
Tipo de venta: Online


In [29]:
df['id'].unique()

array(['...'], dtype=object)

In [31]:
import time
import pandas as pd
import numpy as np

# Establecer ID del cliente objetivo
cliente_id = '1ba596b3179a3d78e58850f5e3bb8fb7957dee11'

# Filtrar transacciones solo de este cliente
datos_cliente = df[df['id'] == cliente_id].copy()  # Usar copy() para evitar SettingWithCopyWarning
datos_cliente['fecha'] = pd.to_datetime(datos_cliente['fecha'])  # Convertir a datetime
datos_cliente = datos_cliente.sort_values('fecha')

# Verificar si hay datos suficientes para el cliente
if len(datos_cliente) < 3:
    print(f"El cliente {cliente_id} tiene muy pocas transacciones ({len(datos_cliente)}) para realizar una predicción confiable.")
else:
    print(f"Analizando {len(datos_cliente)} transacciones del cliente {cliente_id}")

    # Instanciar algoritmo solo para este cliente
    algoritmo_cliente = AlgoritmoPrediccionCompras(datos_cliente)

    # Ejecutar algoritmo
    inicio = time.time()
    prediccion = algoritmo_cliente.ejecutar_para_cliente(cliente_id)
    tiempo_ejecucion = time.time() - inicio

    print(f"\nPredicción completada en {tiempo_ejecucion:.2f} segundos\n")

    # Mostrar resultados
    if prediccion and prediccion['siguiente_fecha'] is not None:
        print(f"RESULTADOS DE PREDICCIÓN PARA {cliente_id}:")
        print(f"---------------------------------------------")
        print(f"Fecha probable de siguiente compra: {prediccion['siguiente_fecha'].strftime('%Y-%m-%d')}")
        print(f"Monto estimado: ${prediccion['siguiente_monto']:.2f}")
        print(f"Tipo de transacción: {'Online' if prediccion['siguiente_tipo'] == 1 else 'Física'}")

        # Calcular la confianza basada en la cantidad de datos y consistencia
        n_transacciones = len(datos_cliente)
        variabilidad_tiempo = datos_cliente['fecha'].diff().dt.days.std()  # Ya convertido a datetime
        confianza = min(95, max(60, 70 + (n_transacciones/20) - (variabilidad_tiempo/10)))

        print(f"Confianza de la predicción: {confianza:.1f}%")

        # Verificar si la fecha de predicción ya pasó la fecha actual
        fecha_actual = pd.to_datetime('2025-05-25')
        if prediccion['siguiente_fecha'] < fecha_actual:
            dias_atraso = (fecha_actual - prediccion['siguiente_fecha']).days
            print(f"\nALERTA: La fecha predicha ya pasó hace {dias_atraso} días.")
            print(f"Es posible que el cliente haya alterado su patrón de compra.")
        elif (prediccion['siguiente_fecha'] - fecha_actual).days <= 7:
            print(f"\nANÁLISIS: Se espera una compra en los próximos 7 días.")
    else:
        print(f"No se pudo generar una predicción confiable para el cliente {cliente_id}")

Analizando 225 transacciones del cliente 1ba596b3179a3d78e58850f5e3bb8fb7957dee11


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.11/dist-packages/scipy/stats/_continuous_distns.py:800: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  a, b = optimize.fsolve(func, (1.0, 1.0))
/usr/local/lib/python3.11/dist-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 


Predicción completada en 2.37 segundos

RESULTADOS DE PREDICCIÓN PARA 1ba596b3179a3d78e58850f5e3bb8fb7957dee11:
---------------------------------------------
Fecha probable de siguiente compra: 2023-01-29
Monto estimado: $31.19
Tipo de transacción: Online
Confianza de la predicción: 81.0%

ALERTA: La fecha predicha ya pasó hace 846 días.
Es posible que el cliente haya alterado su patrón de compra.
